## Web Scraping Watch Product in *jamtangan.com* with Python

### Import Necessary Libraries

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time
import json
import csv
import traceback

from email.message import EmailMessage
import ssl, smtplib

### Function Definition

#### Supporting Function

In [43]:
def to_isoformat(input):
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "Mei": "05",
        "Jun": "06",
        "Jul": "07",
        "Ags": "08",
        "Sep": "09",
        "Okt": "10",
        "Nov": "11",
        "Des": "12"
    }

    for key, value in month_dict.items():
        input = input.replace(key, value)
    
    datetime_object = datetime.strptime(input, "%d %m %Y, %H:%M WIB")

    date = datetime_object.date()
    time = datetime_object.time()

    return [date, time]

def to_email(name):
    name = name.lower().strip()
    email = name.replace(" ", "") + "@gmail.com"
    return email

def csv_to_json(csv_file_path, json_file_path):
    json_array = []

    with open(csv_file_path, encoding='utf-8') as csvf: 
        csv_reader = csv.DictReader(csvf) 

        for row in csv_reader: 
            json_array.append(row)

    with open(json_file_path, 'w', encoding='utf-8') as jsonf: 
        json_string = json.dumps(json_array, indent=4)
        jsonf.write(json_string)

#### Extract Product Table Function

In [44]:
def extract_product(driver, wait):
    product_dict = {
        "Brand" : "",
        "Model No" : "",
        "Series" : "",
        "Gender" : "",
        "Colour" : "",
        "Luminous" : "",
        "Calendar" : "",
        "Water Resistant" : "",
        "Case Diameter (mm)" : "",
        "Strap Material" : "",
    }

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tab-content .grid")))
    spec_grid = driver.find_element(By.CSS_SELECTOR, ".tab-content .grid")
    spec_list = spec_grid.find_elements(By.CSS_SELECTOR, ".spec-item")
    
    for spec in spec_list:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".leading-6")))
        spec_title = spec.find_element(By.CSS_SELECTOR, ".font-black").text
        data = spec.find_elements(By.CSS_SELECTOR, ".leading-6")
        if spec_title in product_dict.keys():
            key = data[0].text.strip()
            value = data[1].text.strip()
            product_dict[key] = value
        elif (spec_title == "Case Diameter"):
            value = float(data[1].text.replace("mm", "").strip())
            product_dict["Case Diameter (mm)"] = value
    
    for (key, value) in product_dict.items():
        if value == "": product_dict[key] = None

    return product_dict

#### Extract Sales Table Function

In [45]:
def extract_sales(driver, wait):
    sales_dict = {
        "Product Name" : "",
        "Brand" : "",
        "Model No" : "",
        "Normal Price" : "",
        "Discounted Price" : "",
        "Discount Percentage" : "",
        "Number of Seen" : "",
        "Number of Sold" : "",
        "Offline Stock Status" : "",
        "Online Stock Status" : "",
    }

    spec_grid = driver.find_element(By.CSS_SELECTOR, ".tab-content .grid")
    spec_list_raw = spec_grid.find_elements(By.CSS_SELECTOR, ".spec-item")
    for spec in spec_list_raw:
        if spec.find_element(By.CSS_SELECTOR, ".font-black").text in ["Brand", "Model No"]:
            data = spec.find_elements(By.CSS_SELECTOR, ".leading-6")
            key = data[0].text.strip()
            value = data[1].text.strip()
            sales_dict[key] = value

    sales_dict["Product Name"] = driver.find_element(By.TAG_NAME, "h1").text.strip()

    try:
        normal_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .line-through").text.strip()
    except:
        normal_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .text-xl").text.strip()
    discounted_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .text-xl").text.strip()
    sales_dict["Normal Price"] = int(normal_price.replace("Rp", "").replace(".", "").strip())
    sales_dict["Discounted Price"] = int(discounted_price.replace("Rp", "").replace(".", "").strip())

    discount_percentage = (sales_dict["Normal Price"] - sales_dict["Discounted Price"]) / sales_dict["Normal Price"]
    sales_dict["Discount Percentage"] = round(discount_percentage * 100, 2)

    num_seen = driver.find_element(By.CSS_SELECTOR, ".ic-eye + div > .text-sm").text.strip()
    if (num_seen.__contains__("Rb")):
        num_seen = float(num_seen.replace(" Rb", "").strip()) * 1000
    sales_dict["Number of Seen"] = int(num_seen)

    num_sold = driver.find_element(By.CSS_SELECTOR, ".ic-cart.mr-1 + div > .text-sm").text.strip()
    if (num_sold.__contains__("Rb")):
        num_sold = float(num_sold.replace(" Rb", "").strip()) * 1000
    sales_dict["Number of Sold"] = int(num_sold)

    try:
        empty_badge = driver.find_element(By.CSS_SELECTOR, ".badge.bg-accent-red")
        if (empty_badge != None and empty_badge.text.strip().__contains__("habis")):
            sales_dict["Online Stock Status"] = "Not Available"
        else:
            raise Exception("The badge is not empty badge")
    except:
        online_stock_status = driver.find_element(By.CSS_SELECTOR, ".stepper-wrapper + div").text.strip()
        if (online_stock_status == "STOK ONLINE < 5 PCS"): sales_dict["Online Stock Status"] = "Low (< 5 PCS)"
        elif (online_stock_status == "STOK ONLINE > 5 PCS"): sales_dict["Online Stock Status"] = "High (>= 5 PCS)"
        else: sales_dict["Online Stock Status"] = "Unknown"

    try:
        offline_empty = driver.find_element(By.CSS_SELECTOR, "picture.mr-2 + div")
        if (offline_empty != None and offline_empty.text.strip().__contains__("Tidak tersedia")):
            sales_dict["Offline Stock Status"] = "Not Available"
        else:
            raise Exception("The text is not empty text")
    except:
        try:
            offline_stock_status = driver.find_element(By.CSS_SELECTOR, "div[data-testid='store-item-0']")
            if (offline_stock_status != None):
                sales_dict["Offline Stock Status"] = "Available"
        except:
            sales_dict["Offline Stock Status"] = "Unknown"

    return sales_dict

#### Extract Customer and Review Table Function

In [46]:
def extract_customer_review(driver, wait):
    customer_dict = {
        "Email" : "",
        "Name" : "",
        "Member Status" : ""
    }
    customer_header = customer_dict.keys()
    customer_list = []

    review_dict = {
        "Product Name" : "",
        "Email" : "",
        "Date" : "",
        "Time" : "",
        "Rating" : "",
        "Delivery Review" : "",
        "Product Review" : ""
    }
    review_header = review_dict.keys()
    review_list = []

    MAX_REVIEW_PER_PRODUCT = 5
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pb-14")))
    div_review_pagination = driver.find_element(By.CSS_SELECTOR, ".pb-14")
    try:
        max_tab_per_product = int(div_review_pagination.find_elements(By.TAG_NAME, "li")[-2].text)
    except:
        max_tab_per_product = 1

    review_count = 0
    tab_count = 0
    # driver.implicitly_wait(10)
    while (review_count < MAX_REVIEW_PER_PRODUCT and tab_count < max_tab_per_product):
        time.sleep(1)
        rating_div = driver.find_elements(By.CSS_SELECTOR, ".bg-neutral-900.col-span-12")[1]
        review_divs = rating_div.find_elements(By.CSS_SELECTOR, ".p-4.relative")

        for review in review_divs:
            reviewer_name = review.find_element(By.CSS_SELECTOR, ".mb-1 .text-base").text
            if (len(reviewer_name) < 3) or (reviewer_name[1] == '*' and reviewer_name[-2] == '*'): continue

            product_name = driver.find_element(By.CSS_SELECTOR, "h1").text

            rating = len(review.find_elements(By.CSS_SELECTOR, ".rating .ic-star-fill"))
            datetime_review = to_isoformat(review.find_element(By.CSS_SELECTOR, "span.block.text-xxs").text)
            date_review, time_review = datetime_review[0], datetime_review[1]
            
            paragraph_review = review.find_elements(By.CSS_SELECTOR, "p")
            delivery_review = ""
            product_review = ""
            for par in paragraph_review:
                title = par.find_element(By.CSS_SELECTOR, "span").text
                if title == "Pengiriman:": delivery_review = par.text.lstrip("Pengiriman: ")
                if title == "Produk:": product_review = par.text.lstrip("Produk: ")
                else: continue
            if delivery_review == "": delivery_review = "Tidak ada review"
            if product_review == "": product_review = "Tidak ada review"

            member_status = driver.find_element(By.CSS_SELECTOR, ".badge.text-xxs").text

            review_count += 1
            if review_count > MAX_REVIEW_PER_PRODUCT: break

            customer_result = [to_email(reviewer_name), reviewer_name, member_status]
            for key, value in zip(customer_header, customer_result):
                customer_dict[key] = value
            customer_list.append(customer_dict.copy())
        
            result_review = [product_name, to_email(reviewer_name), date_review, time_review, rating, delivery_review, product_review]
            for key, value in zip(review_header, result_review):
                review_dict[key] = value
            review_list.append(review_dict.copy())
        
        tab_count += 1
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pb-14")))
        div_review_pagination = driver.find_element(By.CSS_SELECTOR, ".pb-14")
        page_buttons = div_review_pagination.find_elements(By.TAG_NAME, "li")
        for page_button in page_buttons:
            if page_button.text == str(tab_count + 1): 
                page_button.click()
                break

    return [customer_list, review_list]

### Main Program

In [47]:
def main():
    # instantiate the driver and the wait object
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 10)

    # define the URL
    URL = "https://www.jamtangan.com/c/jam-tangan"

    # open the URL
    driver.maximize_window()
    driver.get(URL)

    try:
        # close the pop-up if it exists
        try:
            if (driver.find_element(By.ID, "driver-popover-item")):
                driver.find_element(By.CLASS_NAME, "driver-close-btn").click()

            time.sleep(3)
            if (driver.find_element(By.CSS_SELECTOR, "button.ng-binding")):
                driver.find_element(By.CSS_SELECTOR, "button.ng-binding").click()
        except:
            pass

        # expand the brands list in filter sidebar
        expand_brand = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper .cursor-pointer")[1]
        while(expand_brand.text == "LIHAT LAINNYA"):
            expand_brand.click()
            time.sleep(1)
            expand_brand = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper .cursor-pointer")[1]

        # get the brands name and url that eligible to be scraped
        brands_raw = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper")[1].find_elements(By.TAG_NAME, "li")
        eliminate = ['Semua Brand', 'Band', 'Strap', 'Bracelet', 'Accessories', 'Jewelry', 'Wallets']
        brands = []
        for brand in brands_raw:
            eliminated = False
            for phrase in eliminate:
                if (phrase in brand.find_element(By.TAG_NAME, "label").text):
                    eliminated = True
                    break
            if not eliminated:
                brands.append(brand)
        brands = brands[39:]
        brands_name = [brand.find_element(By.TAG_NAME, "label").text for brand in brands]
        brands_url = [brand.find_element(By.TAG_NAME, "a").get_attribute("href") for brand in brands]

        # declare the container of the tables
        product = []
        sales = []
        customer = []
        review = []

        # scrape the data by iterating each product through the brands
        max_product_per_brand = 20
        product_count = 1
        for brand_url in brands_url:
            driver.get(brand_url)
            wait.until(EC.url_to_be(brand_url))
            time.sleep(1)
            products_tag = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='product-card-test']")[:max_product_per_brand]
            products_url = [product.get_attribute("href") for product in products_tag]
            for product_url in products_url:
                try:
                    driver.get(product_url)
                    wait.until(EC.url_to_be(product_url))
                    print(f"{product_count}. Brand = {brands_name[brands_url.index(brand_url)]} | Product = {product_url}")
                    extracted_product = extract_product(driver, wait)
                    extracted_sales = extract_sales(driver, wait)
                    extracted_customer = extract_customer_review(driver, wait)[0]
                    extracted_review = extract_customer_review(driver, wait)[1]

                    product.append(extracted_product)
                    sales.append(extracted_sales)
                    for item in extracted_customer: 
                        if (item not in customer):
                            customer.append(item)
                    for item in extracted_review: review.append(item)
                    product_count += 1
                except:
                    traceback.print_exc()
                    continue
        driver.close()  
    except:
        traceback.print_exc()
        pass        

    try:
        save_dir = r'../data/'

        # exporting the product table to csv and json
        with open(save_dir + "product.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=product[0].keys())
            writer.writeheader()
            writer.writerows(product)
        product_csv_file_path = save_dir + r'product.csv'
        product_json_file_path = save_dir + r'product.json'
        csv_to_json(product_csv_file_path, product_json_file_path)

        # exporting the sales table to csv and json
        with open(save_dir + "sales.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=sales[0].keys())
            writer.writeheader()
            writer.writerows(sales)
        sales_csv_file_path = save_dir + r'sales.csv'
        sales_json_file_path = save_dir + r'sales.json'
        csv_to_json(sales_csv_file_path, sales_json_file_path)

        # exporting the customer table to csv and json
        with open(save_dir + "customer.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=customer[0].keys())
            writer.writeheader()
            writer.writerows(customer)
        customer_csv_file_path = save_dir + r'customer.csv'
        customer_json_file_path = save_dir + r'customer.json'
        csv_to_json(customer_csv_file_path, customer_json_file_path)

        # exporting the review table to csv and json
        with open(save_dir + "review.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=review[0].keys())
            writer.writeheader()
            writer.writerows(review)
        review_csv_file_path = save_dir + r'review.csv'
        review_json_file_path = save_dir + r'review.json'
        csv_to_json(review_csv_file_path, review_json_file_path)

    except:
        traceback.print_exc()
        pass
    
    print(f"Product Collected = {len(product)}")
    print(f"Sales Collected = {len(sales)}")
    print(f"Customer Collected = {len(customer)}")
    print(f"Review Collected = {len(review)}")

    print("\n========== Scraping is done! ==========\n")

In [48]:
if (__name__ == "__main__"):
    # setup the timer
    time_start = datetime.now()

    # run the main program
    main()

    # setup the timer
    time_end = datetime.now()
    delta = time_end - time_start

    text_wrapper = open(r"Data Scraping\src\config\email_config.txt", "r")

    email_config = text_wrapper.readlines()
    email_to_notify = email_config[0].strip()
    email_password = email_config[1].strip()

    text_wrapper.close()

    subject = "Scraping Jamtangan.com Report"
    body = f"""
    [SCRAPING JAMTANGAN.COM IS COMPLETE]

    Time Start:\t{time_start.strftime('%Y-%m-%d %H:%M:%S')}
    Time End:\t{time_end.strftime('%Y-%m-%d %H:%M:%S')}
    Duration:\t{str(delta).split('.')[0]}

    """

    email = EmailMessage()
    email["From"] = email_to_notify
    email["To"] = email_to_notify
    email["Subject"] = subject
    email.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(email_to_notify, email_password)
        server.sendmail(email_to_notify, email_to_notify, email.as_string())
    
    print(f"Time Start:\t{time_start.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Time End:\t{time_end.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Duration:\t{str(delta).split('.')[0]}\n")

1. Brand = Giordano | Product = https://www.jamtangan.com/p/giordano-easytime-gd213333-men-red-dial-dual-tone-rubber-strap-354661?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Giordano
2. Brand = Giordano | Product = https://www.jamtangan.com/p/giordano-metropolitan-gd106655-men-silver-dial-rainbow-mesh-strap-353271?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Giordano
3. Brand = Giordano | Product = https://www.jamtangan.com/p/giordano-fashionista-gd214422-ladies-rose-gold-dial-dual-tone-mesh-strap-355272?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Giordano
4. Brand = Giordano | Product = https://www.jamtangan.com/p/giordano-eleganza-gd213444-ladies-gold-dial-dual-tone-stainless-steel-strap-354742?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Giordano
5. Brand = Giordano | Product = https://www.jamtangan.com/p/giordano-metropolitan-gd116333-men-white-dial-rose-gold-mesh-strap-354241?list

Traceback (most recent call last):
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\1503922426.py", line 68, in main
    extracted_product = extract_product(driver, wait)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\2703196176.py", line 15, in extract_product
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tab-content .grid")))
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002DA813+48355]
	(No symbol) [0x0026C4B1]
	(No symbol) [0x00175358]
	(No symbol) [0x001A09A5]
	(No symbol) [0x001A0B3B]
	(No symbol) [0x001CE232]
	(No symbol) [0x001BA784]
	(No symbol) [0x001CC922]
	(No symbol) [0x001BA536]
	(No symbol) [0x001982DC]
	(No symbol) [0x001993DD]
	GetHand

269. Brand = MIDO | Product = https://www.jamtangan.com/p/mido-multifort-m038-429-36-061-00-dual-time-automatic-anthracite-dial-brown-leather-strap-246561?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Mido
270. Brand = MIDO | Product = https://www.jamtangan.com/p/mido-commander-m031-631-33-061-00-icone-black-dial-black-mesh-strap-246481?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Mido
271. Brand = MIDO | Product = https://www.jamtangan.com/p/mido-commander-ii-m021-407-33-411-00-gradient-dial-black-stainless-steel-strap-356811?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Mido
272. Brand = MVMT | Product = https://www.jamtangan.com/p/mvmt-voyager-dmv01rggr2-graphite-black-matte-dial-grey-nylon-nato-strap-258111?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Mvmt
273. Brand = MVMT | Product = https://www.jamtangan.com/p/mvmt-element-ion-28000042d-men-grey-dial-grey-leather-strap-385731?list-r

Traceback (most recent call last):
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\1503922426.py", line 71, in main
    extracted_review = extract_customer_review(driver, wait)[1]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\3940597672.py", line 80, in extract_customer_review
    page_button.click()
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 395, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 346, in execute
    self.error_handler.check_response(response)

320. Brand = Orient | Product = https://www.jamtangan.com/p/orient-mako-ii-automatic-divers-faa02002d-men-blue-dial-stainless-steel-strap-139551?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Orient
321. Brand = Orient | Product = https://www.jamtangan.com/p/orient-classic-fac00004b-automatic-black-dial-black-leather-strap-109361?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Orient
322. Brand = Orient | Product = https://www.jamtangan.com/p/orient-bambino-v3-fac0000dd-classic-mechanical-dark-blue-dial-leather-strap-95731?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Orient
323. Brand = Orient | Product = https://www.jamtangan.com/p/orient-bambino-v4-fac08003a-classic-mechanical-grey-dial-brown-leather-strap-86931?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Orient
324. Brand = Orient | Product = https://www.jamtangan.com/p/orient-classic-fag00004d-automatic-skeleton-dial-blue-leather-strap-1

...

612. Brand = Timex | Product = https://www.jamtangan.com/p/timex-waterbury-tw2t70100-automatic-men-black-dial-brown-leather-strap-372911?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Timex
613. Brand = Timex | Product = https://www.jamtangan.com/p/timex-tw5m42200-digital-dial-black-resin-strap-430412?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Timex
614. Brand = Timex | Product = https://www.jamtangan.com/p/timex-dgtl-tw5m41200-digital-dial-black-resin-strap-430371?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Timex
615. Brand = Timex | Product = https://www.jamtangan.com/p/timex-q-tw2u61800-reissue-black-dial-stainless-steel-strap-450281?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Timex
616. Brand = Timex | Product = https://www.jamtangan.com/p/timex-ironman-t5k822-indiglo-digital-dial-black-resin-strap-426401?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Timex
6

Traceback (most recent call last):
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\1503922426.py", line 71, in main
    extracted_review = extract_customer_review(driver, wait)[1]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\3940597672.py", line 80, in extract_customer_review
    page_button.click()
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 395, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 346, in execute
    self.error_handler.check_response(response)

623. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-tsport-t120-407-11-041-03-seastar-1000-powermatic-80-men-blue-dial-stainless-steel-strap-458131?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Tissot


Traceback (most recent call last):
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\1503922426.py", line 71, in main
    extracted_review = extract_customer_review(driver, wait)[1]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fikna\AppData\Local\Temp\ipykernel_20532\3940597672.py", line 80, in extract_customer_review
    page_button.click()
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 395, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\fikna\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 346, in execute
    self.error_handler.check_response(response)

623. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-tsport-t116-617-11-047-01-chrono-xl-classic-blue-dial-stainless-steel-strap-182151?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Tissot
624. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-luxury-powermatic-80-t086-407-11-201-02-black-dial-stainless-steel-strap-361791?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Tissot
625. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-tsport-t116-617-11-057-01-chrono-xl-classic-black-dial-stainless-steel-strap-187311?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Tissot
626. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-pr-100-chronograph-gent-white-dial-stainless-steel-t101-417-22-031-00-93251?list-referer-id=Jam%20Tangan&list-referer-name=Jam%20Tangan%20Brand%20Tissot
627. Brand = TISSOT | Product = https://www.jamtangan.com/p/tissot-tclassic-t109-407-16-031-00-everyt